In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("./balanced_data.csv")
df["Rating"].value_counts(), df["Rating"].value_counts().sum(), df.head(3), df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2835 entries, 0 to 2834
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Rating  2835 non-null   int64 
 1   Review  2835 non-null   object
dtypes: int64(1), object(1)
memory usage: 44.4+ KB


(Rating
 1    567
 2    567
 3    567
 4    567
 5    567
 Name: count, dtype: int64,
 2835,
    Rating                                             Review
 0       1     dont bother the boardwalk is completely broken
 1       1  minim tempah sampah minim nya kesadaran orang ...
 2       1  hotel cocok costumer business travelling ac ge...,
 None)

In [3]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(
    df.Review.values,
    df.Rating.values,
    test_size=0.2,
    random_state=42,
    stratify=df.Rating.values
)

In [4]:
train_distribution = pd.Series(y_train).value_counts()
val_distribution = pd.Series(y_val).value_counts()

print("Training set distribution:")
print(train_distribution)

print("\nValidation set distribution:")
print(val_distribution)

Training set distribution:
2    454
3    454
5    454
4    453
1    453
Name: count, dtype: int64

Validation set distribution:
1    114
4    114
2    113
3    113
5    113
Name: count, dtype: int64


In [7]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False)
max_length = 512

x_train_ids = np.zeros((len(x_train), max_length))
x_train_attmask = np.zeros((len(x_train), max_length))
x_val_ids = np.zeros((len(x_val), max_length))
x_val_attmask = np.zeros((len(x_val), max_length))

train_labels = encoder.fit_transform(y_train.reshape(-1, 1))
val_labels = encoder.transform(y_val.reshape(-1, 1))

x_train_ids.shape, x_val_ids.shape, train_labels.shape, val_labels.shape, train_labels

((2268, 512),
 (567, 512),
 (2268, 5),
 (567, 5),
 array([[0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 1., 0., 0., 0.],
        ...,
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0.]]))

In [8]:
from transformers import BertTokenizer

checkpoint = "indobenchmark/indobert-base-p1"

tokenizer = BertTokenizer.from_pretrained(checkpoint)

def preprocess(texts, max_len):
  input_ids = []
  attention_masks = []
  for text in texts:
    tokenized_text = tokenizer.encode_plus(
        text,
        max_length=max_len,
        truncation=True,
        padding="max_length",
        add_special_tokens=True,
        return_tensors="tf"
    )
    input_ids.append(tokenized_text["input_ids"][0])
    attention_masks.append(tokenized_text["attention_mask"][0])
  return np.array(input_ids), np.array(attention_masks)

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/229k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/home/alridho/miniconda3/envs/capstone/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

In [9]:
x_train_ids, x_train_attmask = preprocess(x_train, max_length)
x_val_ids, x_val_attmask = preprocess(x_val, max_length)

2024-06-15 14:58:04.382761: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-15 14:58:04.610801: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-15 14:58:05.476671: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-15 14:58:07.333971: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-06-15 14:58:08.765593: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/b

In [10]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    x_train_ids,
    x_train_attmask,
    train_labels
    ))

val_dataset = tf.data.Dataset.from_tensor_slices((
    x_val_ids,
    x_val_attmask,
    val_labels
    ))

train_dataset.take(1)

<_TakeDataset element_spec=(TensorSpec(shape=(512,), dtype=tf.int32, name=None), TensorSpec(shape=(512,), dtype=tf.int32, name=None), TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [11]:
def to_dict(input_ids, attention_masks, labels):
  return {
      "input_ids": input_ids,
      "attention_mask": attention_masks
  }, labels

train_dataset = train_dataset.map(to_dict)
val_dataset = val_dataset.map(to_dict)

In [12]:
train_dataset.take(1)

<_TakeDataset element_spec=({'input_ids': TensorSpec(shape=(512,), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(512,), dtype=tf.int32, name=None)}, TensorSpec(shape=(5,), dtype=tf.float64, name=None))>

In [13]:
batch_size = 32

train_dataset = train_dataset.shuffle(1000).batch(batch_size)
val_dataset = val_dataset.batch(batch_size)

In [14]:
train_dataset, val_dataset

(<_BatchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 512), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, 512), dtype=tf.int32, name=None)}, TensorSpec(shape=(None, 5), dtype=tf.float64, name=None))>,
 <_BatchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 512), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, 512), dtype=tf.int32, name=None)}, TensorSpec(shape=(None, 5), dtype=tf.float64, name=None))>)

In [15]:
from transformers import TFAutoModel

indo_bert = TFAutoModel.from_pretrained(checkpoint)

indo_bert.summary()

tf_model.h5:   0%|          | 0.00/656M [00:00<?, ?B/s]

Some layers from the model checkpoint at indobenchmark/indobert-base-p1 were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at indobenchmark/indobert-base-p1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "tf_bert_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  124441344 
                                                                 
Total params: 124441344 (474.71 MB)
Trainable params: 124441344 (474.71 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
input_ids = tf.keras.layers.Input(shape=(max_length,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(max_length,), name='attention_mask', dtype='int32')

x = indo_bert.bert(input_ids, attention_mask=mask)[1]
x = tf.keras.layers.Dense(1024, activation='relu')(x)
y = tf.keras.layers.Dense(5, activation='softmax')(x)

In [17]:
model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)
model.layers[2].trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 512)]                0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 512)]                0         []                            
 )                                                                                                
                                                                                                  
 bert (TFBertMainLayer)      TFBaseModelOutputWithPooli   1244413   ['input_ids[0][0]',           
                             ngAndCrossAttentions(last_   44         'attention_mask[0][0]']      
                             hidden_state=(None, 512, 7                                       

In [18]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
loss = tf.keras.losses.CategoricalCrossentropy()

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=100
)

Epoch 1/100
